In [1]:
from PIL import Image, ImageDraw, ImageFont, ImageOps
from IPython.display import display
import textwrap
Image.MAX_IMAGE_PIXELS = None

from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import requests
import shutil

import os.path
import io
import pickle
from urllib.parse import urlparse
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaIoBaseDownload

In [2]:
SCOPES = ['https://www.googleapis.com/auth/drive']
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
drive = build('drive', 'v3', credentials=creds)

In [4]:
data = pd.read_csv('data/BECMPNA.csv')

In [5]:
def getId(url):
    urls = url.split(',')
    res = []
    for link in urls:
        res.append(urlparse(link).query[3:])
    return res

In [6]:
data['ImageId'] = data['Picture '].apply(lambda x : getId(x))

In [10]:
def generateStudent(target,name,img,quote,roll,index,verbose):
    
    # get image    
    # google api to download file into student.png
    try:
        request = drive.files().get_media(fileId=img)
        fh = io.FileIO("student.png", 'wb')
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            if verbose:
                print(name)  
            
        # File Naming Convention : RollNumber StudentName PhotoNumber Rotation
        filename = str(roll)+' '+name+' '+str(index+1)

        student = Image.open('student.png')
    except Exception as e:
        print(f'Failed on Image : {index+1} Roll : {roll} Name : {name}')
        print(e)
        with open('logs.txt','a') as f:
            f.write(f'Failed on Image : {index+1} Roll : {roll} Name : {name}\n{e}\n\n')
        return

    targetCopy = target.copy()

    if student.size[0]>student.size[1]:
        # save vertical image with 90 shifts
        getImg(target,name,student.rotate(270,expand=True),quote).save(+filename+' 270.png')
        getImg(target,name,student.rotate(90,expand=True),quote).save(filename+' 90.png')
        
    getImg(targetCopy,name,student,quote).save(filename+'.png')
        

def getImg(targetObj,name,student,quote,CORRECTION=7):

    target = targetObj.copy()
    
    d = ImageDraw.Draw(target)
    
    # Paste student image in target
    student.thumbnail( (500,500), Image.ANTIALIAS)
    student = ImageOps.expand(student, border=3)
    target.paste(student,((target.size[0]-student.size[0])//2 + CORRECTION, ((target.size[1]-student.size[1])//2-60 )))
    
    # Add Name
    namefont = ImageFont.truetype("fonts/gabr.ttf", 40)
    w,h = d.textsize(name,namefont)
    d.text(((target.size[0]-w)/2+ CORRECTION,750), name, fill=(0,0,0),font=namefont)
    
    # Add Quote
    quotefont = ImageFont.truetype("fonts/Symbola.ttf", 30)
    currentHt,pad = 810,30
    para = textwrap.wrap(quote, width=45)
    for line in para:
        w,h = d.textsize(line,quotefont)
        d.text(((target.size[0]-w)/2+ CORRECTION,currentHt), line, fill=(0,0,0),font=quotefont)
        currentHt += pad
    
    return target

In [11]:
for i in tqdm(range(data.shape[0])):
    target = Image.open('targets/targetPink.png')
    name = data.iloc[i]['Full Name.']
    img = data.iloc[i]['ImageId']
    quote = data.iloc[i]['Quote']
    roll = data.iloc[i]['Roll No.']
    
    for j in range(len(img)):
        generateStudent(target,name,img[j],quote,roll,j,False)

0%|          | 0/68 [00:03<?, ?it/s]


TypeError: bad operand type for unary +: 'str'